In [72]:
import numpy as np
import scipy

import time
import os

from itertools import combinations, product
from functools import reduce
import copy

import pennylane as qml
from pennylane.templates.layers import RandomLayers
# from pennylane import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

from sklearn.datasets import make_moons

from scipy.stats import spearmanr

# import gym

In [2]:
!pip install pennylane pennylane-qiskit qiskit matplotlib

  Using cached PennyLane-0.25.1-py3-none-any.whl (1.0 MB)
  Using cached PennyLane_qiskit-0.24.0-py3-none-any.whl (34 kB)
  Using cached qiskit-0.38.0-py3-none-any.whl
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached autograd-1.4-py3-none-any.whl (48 kB)
  Using cached PennyLane_Lightning-0.25.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.6 MB)
  Using cached autoray-0.3.2-py3-none-any.whl (36 kB)
  Using cached retworkx-0.11.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
  Using cached mthree-1.0.0-cp37-cp37m-manylinux

## Train random circuits

In [ ]:
x_train = np.genfromtxt('./experiment_data/fmnist_4/x_train.txt')[:, :16]
x_test = np.genfromtxt('./experiment_data/fmnist_4/x_test.txt')[:, :16]

extra_embeds_train = np.zeros((len(x_train), len(x_train)))
extra_embeds_test = np.zeros((len(x_test), len(x_test)))

for i in range(len(x_train[0])):
    extra_embeds_train[:, i] = x_train[:, i] * x_train[:, (i + 1) % len(x_train[0])]
    extra_embeds_test[:, i] = x_test[:, i] * x_test[:, (i + 1) % len(x_train[0])]
    
x_train = np.concatenate((x_train, extra_embeds_train, x_train, extra_embeds_train), 1)
x_test = np.concatenate((x_test, extra_embeds_test, x_test, extra_embeds_test), 1)

y_train = np.genfromtxt('./experiment_data/fmnist_4/y_train.txt')
# y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/fmnist_4/y_test.txt')
# y_test = 1 - 2 * y_test

num_params = [10, 12, 14, 16, 18, 20, 22, 24]

num_qubits = 4
num_embeds = 0

dev = qml.device('lightning.qubit', wires=num_qubits)

for curr_num_params in num_params:
    curr_dir = './random_fe/iqp_2/fmnist_4/{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    for i in range(5):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)
        
        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, curr_num_params, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        extra_circ_gates = ['h'] * num_qubits + ['rz'] * 4 * num_qubits + ['zz'] * 4 * num_qubits 
        circ_gates = extra_circ_gates * 2 + circ_gates
        
        extra_gate_params = [[i] for i in range(num_qubits)] * 5 + [[i, (i + 1) % num_qubits] for i in range(num_qubits)] * 4
        gate_params = extra_gate_params * 2 + gate_params
        
        weights_bounds = [0] * (18 * num_qubits) + weights_bounds
        
        inputs_bounds = [0] * (num_qubits + 1) + [i + 1 for i in range(32)] + [32 for i in range(num_qubits)] +\
            [32 + i + 1 for i in range(32)] + [64 for i in range(len(inputs_bounds) - 1)]
        
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                        weights_bounds, [0], 'exp') 

        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 16000, 0.05, 1, mse_vec_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
            val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list) 

Step 1 | Sliding Loss Window : 1.41465162435344
Step 51 | Sliding Loss Window : 1.4150567340195495
Step 101 | Sliding Loss Window : 1.4616764456510116
Step 151 | Sliding Loss Window : 1.4060469365769726
Step 201 | Sliding Loss Window : 1.498933375786648
Step 251 | Sliding Loss Window : 1.4079955194878795
Step 301 | Sliding Loss Window : 1.4657419759761074
Step 351 | Sliding Loss Window : 1.444721043361109
Step 401 | Sliding Loss Window : 1.410013470632602
Step 451 | Sliding Loss Window : 1.4068582535583305
Step 501 | Sliding Loss Window : 1.4458605967856955
Step 551 | Sliding Loss Window : 1.4337244924033288
Step 601 | Sliding Loss Window : 1.3430110766125531
Step 651 | Sliding Loss Window : 1.391298616869908
Step 701 | Sliding Loss Window : 1.431830623788911
Step 751 | Sliding Loss Window : 1.4865002945034294
Step 801 | Sliding Loss Window : 1.3598597999355622
Step 851 | Sliding Loss Window : 1.4161294379591323
Step 901 | Sliding Loss Window : 1.4399144615284003
Step 951 | Sliding Los

## human design

In [141]:
from importlib import reload

import create_human_design_circs

reload(create_human_design_circs)

<module 'create_human_design_circs' from '/root/create_human_design_circs.py'>

In [ ]:
from datasets_nt import load_dataset
from create_human_design_circs import generate_human_design_circ
from train_circ import mse_loss, mse_vec_loss

import os
import numpy as np

dataset = 'mnist_4'
num_reps = 1

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0, 1]

num_params = [24, 28, 32, 36, 40]
num_embed_layers = 1

num_train_steps = 8000
batch_size = 1
loss_fn = mse_vec_loss

ours_dir = './human_design/amp_basic/mnist_4/'

if not os.path.exists(ours_dir):
    os.mkdir(ours_dir)

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)
x_train = x_train.reshape((len(x_train), 1, 16))
x_test = x_test.reshape((len(x_test), 1, 16))
# x_train = x_train.reshape((len(x_train), num_embed_layers, num_qubits))
# x_test = x_test.reshape((len(x_test), num_embed_layers, num_qubits))

dev = qml.device('lightning.qubit', wires=num_qubits)

var_layer_options = [[qml.RY], [qml.RX]] * 10
enc_layer_options = [[None, True], [None, True]] * 2
# enc_layer_options = [['Y'], ['X']] * 2

for curr_num_params in num_params:
    curr_dir = ours_dir + '{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    curr_num_layers = curr_num_params // num_qubits
    curr_weights_shape = (curr_num_layers, 1, num_qubits)
        
    for i in range(1):
        curr_circ_dir = curr_dir

        circ = generate_human_design_circ(dev, num_qubits, 'amp', 'basic', num_embed_layers, curr_num_layers, enc_layer_options[:num_embed_layers],
                                          var_layer_options[:curr_num_layers], meas_qubits, 'exp')
        
        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, curr_weights_shape, num_train_steps, 0.05, batch_size, loss_fn, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = np.array([circ(x_test[i], info[-1][-1]) for i in range(len(x_test))])
            val_loss = np.array([loss_fn(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

            shape = np.array(info[-1]).shape
            
            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), np.array(info[-1]).reshape(shape[0], np.product(shape[1:])))
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(np.mean(val_loss))
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list) 

Step 1 | Sliding Loss Window : 1.6635749110210971
Step 51 | Sliding Loss Window : 1.477071927551097
Step 101 | Sliding Loss Window : 1.3263816464399523
Step 151 | Sliding Loss Window : 1.1824245655349996
Step 201 | Sliding Loss Window : 1.201723664425429
Step 251 | Sliding Loss Window : 1.1418479427946544
Step 301 | Sliding Loss Window : 1.1019372755122996
Step 351 | Sliding Loss Window : 1.0523647055162455
Step 401 | Sliding Loss Window : 1.0241432299040079
Step 451 | Sliding Loss Window : 1.0423229784411527
Step 501 | Sliding Loss Window : 1.0987647062592698
Step 551 | Sliding Loss Window : 1.0875871248055824
Step 601 | Sliding Loss Window : 0.9987932353606871
Step 651 | Sliding Loss Window : 1.0047765676315241
Step 701 | Sliding Loss Window : 1.0550850726372252
Step 751 | Sliding Loss Window : 1.066249071839468
Step 801 | Sliding Loss Window : 0.9867689401413015
Step 851 | Sliding Loss Window : 1.0109114834229984
Step 901 | Sliding Loss Window : 1.0090284326179921
Step 951 | Sliding

## ours + fixed mebddings

In [ ]:
search_params = [12, 16, 20, 24]

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
ideal_2 = np.concatenate((np.zeros((16, 16)), np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal_2, ideal_2[::-1, :], ideal[::-1, :]), 1)

num_qubits = 4
num_embeds = 0

x_train = np.genfromtxt('./experiment_data/fmnist_4/x_train.txt')[:, :16]
x_train = np.concatenate((x_train, x_train), 1)
x_test = np.genfromtxt('./experiment_data/fmnist_4/x_test.txt')[:, :16]
x_test = np.concatenate((x_test, x_test), 1)

y_train = np.genfromtxt('./experiment_data/fmnist_4/y_train.txt')
# y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/fmnist_4/y_test.txt')
# y_test = 1 - 2 * y_test

class_0_sel = np.random.choice(2000, 16, False)
class_1_sel = np.random.choice(2000, 16, False) + 2000
class_2_sel = np.random.choice(2000, 16, False) + 4000
class_3_sel = np.random.choice(2000, 16, False) + 6000
sel_inds = np.concatenate((class_0_sel, class_1_sel, class_2_sel, class_3_sel))

sel_data = x_train[sel_inds]

np.savetxt('./ours_fe/fmnist_4/angle_2/sel_data.txt', sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_dir = './ours_fe/fmnist_4/angle_2/{}_params'.format(p)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    for i in range(250):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((32, p))
        np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        circ_gates = ['ry'] * 4 + ['rx'] * 4 + ['rz'] * 4 + ['ry'] * 4 + ['ry'] * 4 + ['rx'] * 4 + ['rz'] * 4 + ['ry'] * 4 + circ_gates
        gate_params = [[0], [1], [2], [3]] * 8 + gate_params
        inputs_bounds = [k for k in range(33)] + [32 for i in range(len(inputs_bounds) - 1)]
        weights_bounds = [0 for k in range(32)] + weights_bounds
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)
    
        circ_d2_scores = []
        circ_d2_t_scores = []
    
        for j in range(32):
            curr_params = np.concatenate([params[j] for k in range(64)]).reshape((64, p))
            mat = compute_reduced_similarity(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, curr_params, sel_data, [0, 1])
        
            t_mat = mat > ((np.sum(mat) - 64) / 4032)
        
            diff_mat = mat - ideal
            diff_2d = np.sqrt(np.sum(np.multiply(diff_mat, diff_mat)))
            
            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sqrt(np.sum(np.multiply(diff_t_mat, diff_t_mat)))
            
            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)
            
        np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
        np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
    np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
    np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
    np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
    np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
    np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
    np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)

## train supernet circuits

In [2]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [ ]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'mnist_2'
main_dir = './supernet/mnist_2/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4

device_name = 'ibm_nairobi'
# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [16, 20]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    search_space = len(layer_rots) * len(layer_cnots)
        
    accs = []
    losses = []
    circ_layers = []

    params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
    supernet_device_dir = supernet_dir + '/' + device_name
    
    if not os.path.exists(supernet_device_dir):
        os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)
#         curr_circ_desc = [[272920], [163863], [258091], [172861], [126492]][i]

        curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0], 'exp')
        
        val_exp_list = []
        
        for j in range(len(x_test)):
            val_exp_list.append(circ(x_test[j], curr_params))
            
        val_exps = np.array(val_exp_list).flatten()
        val_losses = np.power(val_exps - y_test, 2)
        val_loss = np.mean(val_losses)
        acc = np.mean(np.multiply(val_exps, y_test) > 0)
            
        print(curr_circ_desc, acc, val_loss)

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
    np.savetxt(supernet_device_dir + '/searched_circ_layers.txt', np.array(circ_layers))
    np.savetxt(supernet_device_dir + '/searched_circ_accs.txt', accs)
    np.savetxt(supernet_device_dir + '/searched_circ_losses.txt', losses)  
#     np.savetxt(supernet_dir + '/searched_circ_accs.txt', ['{} {}'.format(circ_layers[i], accs[i]) for i in range(num_circs)], fmt="%s")

ibmqfactory.load_account:WARNING:2022-10-02 14:01:10,702: Credentials are already in use. The existing account in the session will be replaced.


[206268 299613 330511 294339] 0.485 1.0810948848724364
[187092 271178 124159  74348] 0.5275 0.9956935214996337
[253149 213129 203803 261491] 0.515 0.9991420936584473
[309882  22116  63598 300588] 0.525 1.0223083686828613
[ 92978  35623 259264 311768] 0.49 1.1172869205474854
[287524   1379 237603 207605] 0.4975 1.0228928470611571
[  9840  51239  72267 307684] 0.545 1.0077995109558104
[ 98856 239177 306820 262985] 0.5225 1.0083969593048097
[117413 138267 295149 197860] 0.5025 1.0171551513671875
[149005 263298 157229  37345] 0.5025 1.0406764221191407
[104758  51675 201179 327951] 0.505 1.0787015056610108
[215237 154828 176247 253787] 0.4725 1.1053011894226075
[ 83553 149944  88137 318650] 0.535 1.0137629413604736
[105309 325650 185809 214339] 0.535 1.0080095958709716
[ 52865  75048 109738 290202] 0.4675 1.0574028301239013
[161615  10287 211185 300035] 0.4575 1.0704778003692628
[ 82588 300580  42972 224028] 0.4775 1.0362108993530272
[171215 277858 191364 306289] 0.485 1.0116024684906006
[2

In [ ]:
from datasets import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import mse_vec_loss, train_qnn

dataset = 'fmnist_4'
main_dir = './supernet/fmnist_4/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
meas_qubits = [0, 1]
num_steps = 16000
num_encoding_layers = 4

dev = qml.device('lightning.qubit', wires=num_qubits)

# search_nums = [5, 10, 25, 50, 100, 250]
search_nums = [100]
num_trials = [25]
param_nums = [12, 16, 20, 24]

for p in param_nums:
    curr_params_dir =  main_dir + 'search-{}_params_mb'.format(p)
    
    sc_circs = np.genfromtxt(curr_params_dir + '/searched_circ_accs.txt', dtype=str, delimiter=']')

    circ_layers = [[int(j) for j in i[0][1:].split(' ') if j.isnumeric()] for i in sc_circs]
    circ_accs = [float(i[1]) for i in sc_circs]
    
    circ_layers = np.array(circ_layers)
    circ_accs = np.array(circ_accs)

    layer_gates = [i[1:].split(')') for i in open(curr_params_dir + '/search_space.txt').read().split('\n')][:-1]

    layer_rots = [''.join([j for j in i[0] if j.isupper()]) for i in layer_gates]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = [''.join([j for j in i[1][1:] if j not in ['[', ']', ',', ' ']]) for i in layer_gates]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]

    for k in range(len(search_nums)):
        search_dir = curr_params_dir + '/search_{}'.format(search_nums[k])

        if not os.path.exists(search_dir):
            os.mkdir(search_dir)

        for i in range(num_trials[k]):
            curr_dir = search_dir + '/trial_{}'.format(i + 1)

            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)

            chosen_circs = range(search_nums[k] * i, search_nums[k] * (i + 1))
            best_subnet = circ_layers[chosen_circs[np.argmax(circ_accs[chosen_circs])]]

            np.savetxt(curr_dir + '/sampled_circuits.txt', circ_layers[chosen_circs])
            np.savetxt(curr_dir + '/sampled_circuit_accs.txt', circ_accs[chosen_circs])

            circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(best_subnet, num_encoding_layers,
                                                                                                          layer_rots, layer_cnots, num_qubits)
            
            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                            weights_bounds, meas_qubits, 'exp', 'adjoint') 

            losses_list = []
            accs_list = []

            for j in range(5):
                curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], num_steps, 0.05, 1, mse_vec_loss, verbosity=17300, 
                                                                                                loss_window=50, init_params=None, 
                                                                                                acc_thres=1.1, shuffle=True, print_loss=50)

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                losses_list.append(val_loss)
                accs_list.append(acc)

            np.savetxt(curr_dir + '/accs.txt', accs_list)
            np.savetxt(curr_dir + '/val_losses.txt', losses_list)            

            np.savetxt(curr_dir + '/gates.txt', circ_gates, fmt="%s")
            np.savetxt(curr_dir + '/gate_params.txt', gate_params, fmt="%s")
            np.savetxt(curr_dir + '/inputs_bounds.txt', inputs_bounds)
            np.savetxt(curr_dir + '/weights_bounds.txt', weights_bounds)
            np.savetxt(curr_dir + '/layers.txt', best_subnet)

Step 1 | Sliding Loss Window : 1.4312615703491827
Step 51 | Sliding Loss Window : 1.3264771503863988
Step 101 | Sliding Loss Window : 1.3775252920050338
Step 151 | Sliding Loss Window : 1.3971212527004189
Step 201 | Sliding Loss Window : 1.477749731001871
Step 251 | Sliding Loss Window : 1.4462158568170589
Step 301 | Sliding Loss Window : 1.3695376825979724
Step 351 | Sliding Loss Window : 1.4072434223407
Step 401 | Sliding Loss Window : 1.3271691907061784
Step 451 | Sliding Loss Window : 1.3826290481509682
Step 501 | Sliding Loss Window : 1.3832443216037615
Step 551 | Sliding Loss Window : 1.4905651161246554
Step 601 | Sliding Loss Window : 1.4255001374382297
Step 651 | Sliding Loss Window : 1.3587562812041716
Step 701 | Sliding Loss Window : 1.3364947563532315
Step 751 | Sliding Loss Window : 1.4075202614641986
Step 801 | Sliding Loss Window : 1.4102981123195486
Step 851 | Sliding Loss Window : 1.3647270990920197
Step 901 | Sliding Loss Window : 1.414645207931733
Step 951 | Sliding L

## ours + fixed embedding

In [ ]:
from datasets import load_dataset
from create_gate_circs import create_batched_gate_circ
from metrics import compute_reduced_similarity

search_params = [2, 4, 6, 8, 10, 12, 14, 16]

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

num_qubits = 2
num_embeds = 0
meas_qubits = [0]
num_param_runs = 32

x_train, y_train, x_test, y_test = load_dataset('moons_300', 'angle', 2)
main_dir = './ours_fe/moons/angle_2'

class_0_sel = np.random.choice(240, 16, False)
class_1_sel = np.random.choice(240, 16, False) + 240
sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]
num_data = len(sel_data)

np.savetxt(main_dir + '/sel_data.txt', sel_data)

dev = qml.device('lightning.qubit', wires=num_qubits)

for p in search_params:
    curr_dir = main_dir + '/{}_params'.format(p)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
    
    d2_min_scores = []
    d2_mean_scores = []
    d2_var_scores = []
    
    d2_t_min_scores = []
    d2_t_mean_scores = []
    d2_t_var_scores = []
    
    mean_mat_scores = []
    mean_t_mat_scores = []

    for i in range(2500):
        curr_circ_dir = curr_dir + '/circ_{}'.format(i + 1)

        if not os.path.exists(curr_circ_dir):
            os.mkdir(curr_circ_dir)
        
        ent_prob = np.random.sample()
        cxz = np.random.sample()
        pauli = 0

        params = 2 * np.pi * np.random.sample((num_param_runs, p))
        np.savetxt(curr_circ_dir + '/params.txt', params)
        
        circ_gates, gate_params, inputs_bounds, weights_bounds = generate_true_random_gate_circ(num_qubits, num_embeds, p, 
                                                                                                        ent_prob=ent_prob, 
                                                                                                        cxz_prob=cxz * ent_prob,
                                                                                                        pauli_prob=pauli * (
                                                                                                            1 - cxz) * ent_prob)
        
        circ_gates = ['ry'] * 2 + ['rx'] * 2 + circ_gates
        gate_params = [[0], [1], [0], [1]]+ gate_params
        inputs_bounds = [0, 1, 2, 3] + [4 for i in range(len(inputs_bounds))]
        weights_bounds = [0 for i in range(4)] + weights_bounds
        
        np.savetxt(curr_circ_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_circ_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_circ_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_circ_dir + '/weights_bounds.txt', weights_bounds)

        batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                            weights_bounds, meas_qubits, 'matrix') 

        circ_d2_scores = []
        circ_d2_t_scores = []
        circ_mean_mat = np.zeros((num_data, num_data))
        circ_t_mean_mat = np.zeros((len(sel_data), len(sel_data)))

        for j in range(num_param_runs):
            curr_params = np.concatenate([params[j] for k in range(num_data)]).reshape((num_data, p))
            mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
            
            t_mat = mat > ((np.sum(mat) - num_data) / (num_data * (num_data - 1)))
        
            diff_mat = mat - ideal
            diff_2d = np.sum(np.multiply(diff_mat, diff_mat))

            diff_t_mat = t_mat - ideal
            diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))

            circ_d2_scores.append(diff_2d)
            circ_d2_t_scores.append(diff_2d_t)

            circ_mean_mat += mat / num_param_runs
            circ_t_mean_mat += t_mat / num_param_runs
            
        np.savetxt(curr_circ_dir + '/d2_scores.txt', circ_d2_scores)
        np.savetxt(curr_circ_dir + '/d2_t_scores.txt', circ_d2_t_scores)
        
        d2_min_scores.append(np.min(circ_d2_scores))
        d2_mean_scores.append(np.mean(circ_d2_scores))
        d2_var_scores.append(np.var(circ_d2_scores))
        
        d2_t_min_scores.append(np.min(circ_d2_t_scores))
        d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
        d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
        diff_mean_mat = ideal - circ_mean_mat
        diff_t_mean_mat = ideal - circ_t_mean_mat
        
        mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
        mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
        
        print(i)
        
    np.savetxt(curr_dir + '/d2_mean_scores.txt', d2_mean_scores)
    np.savetxt(curr_dir + '/d2_min_scores.txt', d2_min_scores)
    np.savetxt(curr_dir + '/d2_var_scores.txt', d2_var_scores)
    
    np.savetxt(curr_dir + '/d2_t_mean_scores.txt', d2_t_mean_scores)
    np.savetxt(curr_dir + '/d2_t_min_scores.txt', d2_t_min_scores)
    np.savetxt(curr_dir + '/d2_t_var_scores.txt', d2_t_var_scores)
    
    np.savetxt(curr_dir + '/d2_mean_mat_scores.txt', mean_mat_scores)
    np.savetxt(curr_dir + '/d2_mean_t_mat_scores.txt', mean_t_mat_scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## train our circuits

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import get_circ_params, create_gate_circ
from train_circ import train_qnn, mse_loss
from create_noise_models import noisy_dev_from_backend

search_nums = [100]
param_nums = [4, 8, 12, 16, 20]

dataset = 'bank'
num_reps = 2

num_qubits = 4
dev = qml.device('lightning.qubit', wires=num_qubits)
meas_qubits = [0]
mat_size = 1024

device_name = 'ibmq_lima'
noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

ours_dir = './ours/bank/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', num_reps)

for search_num in search_nums:       
    for param_num in param_nums:
        noise_scores = []
        
        for i in range(2500):
            noise_scores.append(np.genfromtxt(ours_dir + '/{}_params/circ_{}/noise_metric/{}/metric_tvd_score.txt'.format(param_num, i + 1, device_name)))
            
        noise_scores = np.array(noise_scores)
        
        mean_t_mat_scores = np.genfromtxt(ours_dir + '{}_params/d2_mean_t_mat_scores.txt'.format(param_num))
        perf_scores = (mat_size - mean_t_mat_scores) / mat_size
        combined_scores = np.multiply(perf_scores, noise_scores)
        
        circ_inds = np.random.permutation(2500)
        
        curr_dir = ours_dir + '/{}_params/search_{}_{}'.format(param_num, search_num, device_name)
        
        if not os.path.exists(curr_dir):
            os.mkdir(curr_dir)

        for j in range(25):
            curr_trial_dir = curr_dir + '/trial_{}'.format(j + 1)

            if not os.path.exists(curr_trial_dir):
                os.mkdir(curr_trial_dir)
        
            sel_inds = circ_inds[range(j * 100, j * 100 + 100)]
            sel_ind = sel_inds[np.argmax(combined_scores[sel_inds])]

            np.savetxt(curr_trial_dir + '/searched_circuit_inds.txt', sel_inds)
            np.savetxt(curr_trial_dir + '/searched_circuit_scores.txt', combined_scores[sel_inds])
            np.savetxt(curr_trial_dir + '/sel_circuit_ind.txt', [sel_ind])
            np.savetxt(curr_trial_dir + '/sel_circuit_score.txt', [combined_scores[sel_ind]])
            
            print(sel_inds, combined_scores[sel_inds])
            print(sel_ind, combined_scores[sel_ind])
            print(noise_scores[sel_ind], perf_scores[sel_ind])

            circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(ours_dir + '{}_params/circ_{}'.format(param_num, sel_ind + 1))

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint')
            
            noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', None)
        
            losses_list = []
            accs_list = []
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_trial_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], 5490, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                loss_window=50, init_params=None, 
                                                                                                acc_thres=1.1, shuffle=True, print_loss=50)

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                
                noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                noisy_val_loss = np.array([mse_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)
                noisy_acc = np.mean(noisy_val_loss < 1)
#                 acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])             

                losses_list.append(val_loss)
                accs_list.append(acc)
                
                noisy_losses_list.append(noisy_val_loss)
                noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_trial_dir + '/accs.txt', accs_list)
            np.savetxt(curr_trial_dir + '/val_losses.txt', losses_list)          
            np.savetxt(curr_trial_dir + '/noisy_accs.txt', noisy_accs_list)
            np.savetxt(curr_trial_dir + '/noisy_val_losses.txt', noisy_losses_list) 

[ 162  751  965 2384 1016 1907 1153  830   15  988  246 2205  766  315
 1457   46 1737  327 1389 1027 1080  359 2092  782 1482 2153 1984  440
  247 1848 1764 1003   50  144  253  752  287 2121 2069 1255 1455  829
  872 1599 1537 1403 1739  156 1889 1762  300  734  904   59 2171  919
  946 1807 2454 1688 1515  140  466  861 1998   89 2197 2156 1562  304
  862 2091  511  504  299 1607  924 1004  209 1751  968 1060 1622  718
  796  101  936 1342  801 1249  771 2037 1264 2440   18  555 1625 2018
  207 1119] [0.46924818 0.55942282 0.53503928 0.65771142 0.57985538 0.52732685
 0.47242663 0.52731189 0.47061512 0.57632589 0.50185784 0.49787921
 0.48410344 0.49685669 0.51131988 0.54587411 0.61782564 0.48349717
 0.48422241 0.49951458 0.48587543 0.63289607 0.44777957 0.4864428
 0.4759879  0.45713425 0.60635831 0.68527621 0.50500202 0.67464849
 0.52332687 0.46901706 0.45986176 0.56713578 0.48421115 0.57125227
 0.59333773 0.62705725 0.58779864 0.56912966 0.59939256 0.47294998
 0.53630869 0.46727586 

In [151]:
from importlib import reload

import datasets_nt

reload(datasets_nt)

<module 'datasets_nt' from '/root/datasets_nt.py'>

## Metics for correlation circuits

In [ ]:
from create_gate_circs import generate_random_gate_circ, generate_true_random_gate_circ, create_gate_circ, create_batched_gate_circ, get_circ_params
from create_human_design_circs import generate_human_design_circ
from metrics import compute_reduced_similarity
from datasets_nt import load_dataset

ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
ideal_2 = np.concatenate((np.zeros((16, 16)), np.ones((16, 16)), np.zeros((16, 16)), np.zeros((16, 16))))
ideal = np.concatenate((ideal, ideal_2, ideal_2[::-1, :], ideal[::-1, :]), 1)

# ideal = np.concatenate((np.ones((16, 16)), np.zeros((16, 16))))
# ideal = np.concatenate((ideal, ideal[::-1, :]), 1)

# ideal = 2 * ideal - 1

dataset = 'vowel_4'

curr_dir = f'./experiment_data/{dataset}/trained_circuits'
num_qubits = 4
meas_qubits = [0, 1]
num_samples_per_class = 150

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

class_0_sel = np.random.choice(num_samples_per_class, 16, False)
class_1_sel = np.random.choice(num_samples_per_class, 16, False) + num_samples_per_class
class_2_sel = np.random.choice(num_samples_per_class, 16, False) + num_samples_per_class * 2
class_3_sel = np.random.choice(num_samples_per_class, 16, False) + num_samples_per_class * 3
sel_inds = np.concatenate((class_0_sel, class_1_sel, class_2_sel, class_3_sel))

# class_0_sel = np.random.choice(len(x_test) // 2, 16, False)
# class_1_sel = np.random.choice(len(x_test) // 2, 16, False) + (len(x_test) // 2)
# sel_inds = np.concatenate((class_0_sel, class_1_sel))

sel_data = x_train[sel_inds]
# sel_data = np.genfromtxt(curr_dir + '/sel_data.txt')

num_params = 32
num_data = len(sel_data)

d2_min_scores = []
d2_mean_scores = []
d2_var_scores = []
    
d2_t_min_scores = []
d2_t_mean_scores = []
d2_t_var_scores = []

mean_mat_scores = []
mean_t_mat_scores = []

np.savetxt(curr_dir + '/sel_data.txt', sel_data)

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)
    
    batched_circ = create_batched_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                        weights_bounds, meas_qubits, 'matrix') 
    
    params = 2 * np.pi * np.random.sample((num_params, weights_bounds[-1]))
    
    if not os.path.exists(circ_dir + '/fid_mats'):
        os.mkdir(circ_dir + '/fid_mats')
    
    np.savetxt(circ_dir + '/fid_mats/metric_params.txt', params) 
    
    circ_d2_scores = []
    circ_d2_t_scores = []
    circ_mean_mat = np.zeros((num_data, num_data))
    circ_t_mean_mat = np.zeros((num_data, num_data))
    
    for j in range(num_params):
        curr_params = np.concatenate([params[j] for k in range(num_data)]).reshape((num_data, weights_bounds[-1]))
        mat = compute_reduced_similarity(batched_circ, curr_params, sel_data)
        
        t_mat = mat > ((np.sum(mat) - num_data) / (num_data * (num_data - 1)))
        
        diff_mat = mat - ideal
        diff_2d = np.sum(np.multiply(diff_mat, diff_mat))
            
        diff_t_mat = t_mat - ideal
        diff_2d_t = np.sum(np.multiply(diff_t_mat, diff_t_mat))
            
        circ_d2_scores.append(diff_2d)
        circ_d2_t_scores.append(diff_2d_t)
            
        circ_mean_mat += mat / num_params
        circ_t_mean_mat += t_mat / num_params

    np.savetxt(circ_dir + '/fid_mats/d2_scores_2.txt', circ_d2_scores)
    np.savetxt(circ_dir + '/fid_mats/d2_t_scores_2.txt', circ_d2_t_scores)
        
    d2_min_scores.append(np.min(circ_d2_scores))
    d2_mean_scores.append(np.mean(circ_d2_scores))
    d2_var_scores.append(np.var(circ_d2_scores))
        
    d2_t_min_scores.append(np.min(circ_d2_t_scores))
    d2_t_mean_scores.append(np.mean(circ_d2_t_scores))
    d2_t_var_scores.append(np.var(circ_d2_t_scores))
        
    diff_mean_mat = ideal - circ_mean_mat
    diff_t_mean_mat = ideal - circ_t_mean_mat   
    
    mean_mat_scores.append(np.sum(np.multiply(diff_mean_mat, diff_mean_mat)))
    mean_t_mat_scores.append(np.sum(np.multiply(diff_t_mean_mat, diff_t_mean_mat)))
    
    np.savetxt(circ_dir + '/fid_mats/mean_t_mat.txt', circ_t_mean_mat)
    
    print(i)
        
np.savetxt(curr_dir + '/d2_mean_scores_2.txt', d2_mean_scores)
np.savetxt(curr_dir + '/d2_min_scores_2.txt', d2_min_scores)
np.savetxt(curr_dir + '/d2_var_scores_2.txt', d2_var_scores)
    
np.savetxt(curr_dir + '/d2_t_mean_scores_2.txt', d2_t_mean_scores)
np.savetxt(curr_dir + '/d2_t_min_scores_2.txt', d2_t_min_scores)
np.savetxt(curr_dir + '/d2_t_var_scores_2.txt', d2_t_var_scores)
    
np.savetxt(curr_dir + '/d2_mean_mat_scores_kta.txt', mean_mat_scores)
np.savetxt(curr_dir + '/d2_mean_t_mat_scores_kta.txt', mean_t_mat_scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## noise mrtric for correlation circuits

In [100]:
from importlib import reload

import create_gate_circs

reload(create_gate_circs)

<module 'create_gate_circs' from '/root/create_gate_circs.py'>

In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'fmnist_4'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './ours/{}'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

param_nums = [12, 16, 20, 24]

num_cdcs = 128
num_shots = 1024

device_name = 'ibmq_lima'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for p in param_nums:
    param_dir = curr_dir + '/{}_params'.format(p)
    
    for i in range(2500):
        circ_dir = param_dir + '/circ_{}'.format(i + 1)

        circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

        noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                    weights_bounds, meas_qubits, 'probs') 

        noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                    weights_bounds, meas_qubits, 'probs') 

        noise_metric_dir = circ_dir + '/noise_metric'

        if not os.path.exists(noise_metric_dir):
            os.mkdir(noise_metric_dir)

        device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

        if not os.path.exists(device_noise_metric_dir):
            os.mkdir(device_noise_metric_dir)

        params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
        batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]

        noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
        noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

        actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))

        tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)

        np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
        np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])

        score_tvds.append(1 - tvd)
        actual_tvds.append(actual_tvd)

        print(1 - tvd, actual_tvd, i)

ibmqfactory.load_account:WARNING:2022-09-20 04:06:03,887: Credentials are already in use. The existing account in the session will be replaced.


0.8391647338867188 0.8503189086914062 0
0.8915328979492188 0.898223876953125 1
0.8729400634765625 0.8775177001953125 2
0.93487548828125 0.9338531494140625 3
0.9244155883789062 0.9232864379882812 4
0.8988113403320312 0.900421142578125 5
0.9113006591796875 0.9130020141601562 6
0.927764892578125 0.9193649291992188 7
0.927490234375 0.9240188598632812 8
0.8939590454101562 0.8986968994140625 9
0.874267578125 0.8768844604492188 10
0.9225311279296875 0.9222869873046875 11
0.671142578125 0.6852645874023438 12
0.903900146484375 0.9107666015625 13
0.9165115356445312 0.9256134033203125 14
0.9096527099609375 0.9186630249023438 15
0.8458328247070312 0.8444671630859375 16
0.8897781372070312 0.8967361450195312 17
0.9228973388671875 0.923187255859375 18
0.8547744750976562 0.856475830078125 19
0.8531570434570312 0.8885574340820312 20
0.8703689575195312 0.878814697265625 21
0.9186553955078125 0.916900634765625 22
0.8255538940429688 0.7762908935546875 23
0.9184799194335938 0.9155654907226562 24
0.83807373

In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'fmnist_4'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './experiment_data/{}/trained_circuits'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

num_cdcs = 128
num_shots = 1024

device_name = 'ibmq_quito'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 

    noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 
    
    noise_metric_dir = circ_dir + '/noise_metric'
    
    if not os.path.exists(noise_metric_dir):
        os.mkdir(noise_metric_dir)
    
    device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

    if not os.path.exists(device_noise_metric_dir):
        os.mkdir(device_noise_metric_dir)
    
    params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
    batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]
    
    noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
    noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

    actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))
    
    tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)
    
    np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
    np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])
    
    score_tvds.append(1 - tvd)
    actual_tvds.append(actual_tvd)
    
    print(1 - tvd, actual_tvd, i)

# np.savetxt(curr_dir + '/noise_scores.txt', score_tvds)
# np.savetxt(curr_dir + '/noisy_fids.txt', actual_tvds)

ibmqfactory.load_account:WARNING:2022-10-24 03:44:10,722: Credentials are already in use. The existing account in the session will be replaced.


0.840423583984375 0.76226806640625 0
0.8954925537109375 0.9072723388671875 1
0.8821640014648438 0.8843536376953125 2
0.7919998168945312 0.85406494140625 3
0.8474655151367188 0.8592605590820312 4
0.8794403076171875 0.8782196044921875 5
0.7962875366210938 0.8416976928710938 6
0.8749313354492188 0.88543701171875 7
0.8699951171875 0.8838729858398438 8
0.8701019287109375 0.8886337280273438 9
0.8292694091796875 0.8642425537109375 10
0.8798904418945312 0.9059829711914062 11
0.8453292846679688 0.8646621704101562 12
0.8987960815429688 0.9087142944335938 13
0.7358322143554688 0.7660293579101562 14
0.8975448608398438 0.8990097045898438 15
0.8195571899414062 0.83642578125 16
0.823760986328125 0.85467529296875 17
0.87890625 0.8875045776367188 18
0.857757568359375 0.8743896484375 19
0.8576583862304688 0.8663787841796875 20
0.8899459838867188 0.89520263671875 21
0.8260345458984375 0.865264892578125 22
0.8624267578125 0.8805007934570312 23
0.8805007934570312 0.8833389282226562 24
0.85400390625 0.87151

## Trai correlation circuits

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'mnist_2'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 32
num_params = 12

device_name = 'ibm_nairobi'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(400, 600):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

#             acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

ibmqfactory.load_account:WARNING:2022-10-21 17:38:32,159: Credentials are already in use. The existing account in the session will be replaced.


1.0978491207517789 1.0472831077575684 401


In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_vec_loss

import pennylane as qml
import numpy as np
import os

dataset = 'mnist_4'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

print(x_train.shape)

num_qubits = 4
num_embeds = 16
num_params = 32

for i in range(660, 780):
    curr_dir = './experiment_data/mnist_4/trained_circuits/circ_{}'.format(i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(qml.device('lightning.qubit', wires=num_qubits), circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0, 1], 'exp')    

    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        
        info = train_qnn(circ, x_train, y_train, x_test, y_test, [num_params], 16000, 0.05, 1, mse_vec_loss, verbosity=127300, 
                                                                                        loss_window=50, init_params=None, 
                                                                                        acc_thres=1.1, shuffle=True, print_loss=50)
        
        val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
        val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

        acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
        
        np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
        np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])
        
        losses_list.append(val_loss)
        accs_list.append(acc)
        
    np.savetxt(curr_dir + '/val_losses.txt', losses_list)
    np.savetxt(curr_dir + '/accs.txt', accs_list)

(8000, 16)
Step 1 | Sliding Loss Window : 1.3755712691060438
Step 51 | Sliding Loss Window : 1.3263050482928733
Step 101 | Sliding Loss Window : 1.2774119869151668
Step 151 | Sliding Loss Window : 1.260539367633448
Step 201 | Sliding Loss Window : 1.1650949181177632
Step 251 | Sliding Loss Window : 1.1594429231939567
Step 301 | Sliding Loss Window : 1.1217299809705275
Step 351 | Sliding Loss Window : 1.0926516135174242


In [ ]:
from datasets import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import train_qnn, mse_loss

dataset = 'mnist_2'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 32
num_params = 12

dev = qml.device('lightning.qubit', wires=num_qubits)
# dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(125, 130):
    curr_dir = './experiment_data/mnist_2/trained_circuits/circ_{}'.format(i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')    

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        
        info = train_qnn(circ, x_train, y_train, x_test, y_test, [num_params], 3200, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                        loss_window=50, init_params=None, 
                                                                                        acc_thres=1.1, shuffle=True, print_loss=50)
        
        val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
        val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

        acc = np.mean(val_loss < 1)
        
        np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
        np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])
        
        losses_list.append(val_loss)
        accs_list.append(acc)
        
    np.savetxt(curr_dir + '/val_losses.txt', losses_list)
    np.savetxt(curr_dir + '/accs.txt', accs_list)

Step 1 | Sliding Loss Window : 1.157386971785909
Step 51 | Sliding Loss Window : 0.9436126395984995
Step 101 | Sliding Loss Window : 0.7359753301148625
Step 151 | Sliding Loss Window : 0.7571551065541918
Step 201 | Sliding Loss Window : 0.5514913446855106
Step 251 | Sliding Loss Window : 0.6184735928443386
Step 301 | Sliding Loss Window : 0.6485439335969018
Step 351 | Sliding Loss Window : 0.6042994789843836
Step 401 | Sliding Loss Window : 0.5698446956739474
Step 451 | Sliding Loss Window : 0.5215776578862934
Step 501 | Sliding Loss Window : 0.5791680500047712
Step 551 | Sliding Loss Window : 0.5040527390144974
Step 601 | Sliding Loss Window : 0.5767011934254925
Step 651 | Sliding Loss Window : 0.46428982363658294
Step 701 | Sliding Loss Window : 0.49833745655167616
Step 751 | Sliding Loss Window : 0.4510229637699292
Step 801 | Sliding Loss Window : 0.43184593788282477
Step 851 | Sliding Loss Window : 0.45517681245732916
Step 901 | Sliding Loss Window : 0.4919886759573158
Step 951 | S